# Name: Mohammad Javad Noroozi
# Student Number: 99102434
# Project

### import the necessary library

In [31]:
# !pip install pandas

In [32]:
import json
import datetime
import random
import pandas as pd


In [33]:
# For measuring the total running time of this notebook
start_notebook_t = datetime.datetime.now()
print(start_notebook_t)

2024-02-06 04:15:47.671305


In [34]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext


### Read data

In [35]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")


### Downsampling (If your RAM memory is low)

In [36]:
# tweets_rdd = tweets_rdd.sample(False, 0.5, seed=42)


### Show a sample of twitts

In [37]:
# tweets_rdd.takeSample(False, 1, 19)

### Show size of data (No. tweets)

In [38]:
# tweets_rdd.count()

### Select a User

## Extracting the selected user's favorite tweet ids

### Define Parameters for the Primary Rating.

In [39]:
def extract_edge(x):
    tweet = json.loads(x)
    u1 = tweet['user']['id']
    ver = int(tweet['user']['verified'])
    f = int(tweet['user']['followers_count']/1000)
    lc = int(tweet['user']['listed_count']/10)

    if tweet['tweet_type'] == 'generated':
        return [((u1, ver), ((0,0,0,1,f,lc), set()))]

    u2 =[
        tweet.get('in_reply_to_user_id_str', None),
        tweet.get('quoted_status', {}).get('user', {}).get('id'),
        tweet.get('retweeted_status', {}).get('user', {}).get('id'),
    ]
    u2 = [i for i in u2 if i is not None]
    if not u2:
        return[]
    
    u2 = {u2[0]}    

    status = (
        int(tweet['tweet_type'] == 'retweeted'),
        int(tweet['tweet_type'] == 'replied'),
        int(tweet['tweet_type'] == 'quoted'),
        0,
        f,
        lc,
    )

    return [((u1, ver), (status, u2))]


def cal_vote(ver, status, length):
    '''
    The output indicates that this user can choose the number of other users.
    '''
    ret, rep, q, o, f, lc = status

    return int((0.5 * ret + 0.2 * rep + q + ver * length * 0.4 + f + lc) * o/sum(status)*20)


def random_walk(x):
    # x: [((u1, ver), ((ret, rep, q, o, f, lc), {u2s}))]

    (u1, ver), (status, u2s) = x
    if len(u2s) < 3:
        return []
    
    u2s = list(u2s)
    vote = cal_vote(ver, status, len(u2s))
    value = vote/len(u2s) + ver
    N = vote  # N times start random walk from begining
    scores = {u1: value}
    for _ in range(N):
        u2 = random.choice(u2s)
        scores[u2] = scores.get(u2, 0) + value
    return list(scores.items())
           

In [40]:
# [((u1, ver), ((ret,rep,q,o,f,lc), {u2}))]
edge_rdd = tweets_rdd.flatMap(extract_edge)

# [((u1, ver), ((ret,rep,q,o,f,lc), {u2}))]
edges_rdd = edge_rdd.reduceByKey(
    lambda x, y: 
    (
        (
            x[0][0]+y[0][0],
            x[0][1]+y[0][1],
            x[0][2]+y[0][2],
            x[0][3],
            x[0][4]+y[0][4],
            x[0][5]+y[0][5],
        ),
        x[1].union(y[1]),
    )
)

th = 150
# (uid, score)
scores_rdd = edges_rdd.flatMap(random_walk)\
    .reduceByKey(lambda x, y: x + y)\
    .filter(lambda x: x[1] > th)




Py4JJavaError: An error occurred while calling o88.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/javad/venvspark/notebooks/Project/twitter_data_v2.jsonl
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.api.java.JavaRDDLike.partiti

In [ ]:
scores = scores_rdd.collect()

In [ ]:
uids, s = zip(*scores)

In [ ]:
sorted(s)[-100:]
# scores[:100]

[150.08278027111135,
 150.86775592828226,
 152.7810449528386,
 152.90036029436104,
 154.3125159811317,
 154.55383533133536,
 155.4155096433352,
 155.96857192266657,
 156.64875587757805,
 157.3227492314449,
 158.3115892247379,
 160.3501408940504,
 160.92349790448253,
 160.9886603776631,
 161.4381412267193,
 163.8075171253657,
 163.97120685680594,
 164.05115688061895,
 167.43051082827276,
 167.4848673001408,
 168.52812963707777,
 170.0614125659019,
 172.12674364215135,
 172.75349115255923,
 173.3596623487745,
 174.4957270543782,
 174.85728104847098,
 175.47777752637253,
 179.25496607782895,
 183.77999700242086,
 185.27483115124136,
 186.07927796541406,
 190.09874746066697,
 194.58602947137663,
 196.43992141338532,
 198.56679328296974,
 198.90219794521954,
 200.16810089157087,
 200.21361924538544,
 200.950861987184,
 202.38071420009243,
 203.41024979922042,
 204.91928868022094,
 206.23756532285995,
 208.38767507002802,
 208.94107418515887,
 211.53005042867778,
 221.99288002498778,
 236.10

In [ ]:
necessary_keys = ['tweet_type', 'user', 'id','text', 'name', 'screen_name', 
                'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
                'quoted_status', 'retweeted_status','profile_image_url',
                "tweet_type",
                "view_count",
                "twitter_engagement_rate",
                "retweet_count",
                "reply_count",
                "reach",
                "quote_count",
                "influence",
                "impression_rate",
                "impression",
                "favorited",
                "favorite_count",
                "engagement_rate",
                "engagement",
                 ]
top_tweets = tweets_rdd.filter(lambda x: json.loads(x)['user']['id'] in uids and json.loads(x)['tweet_type'] == 'generated')\
                            .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                            .collect()



In [ ]:
user_statistics_keys = [
    "name",
    "verified",
    "followers_count",
    "friends_count",
    "listed_count",
    "favourites_count",
    "statuses_count",
]

tweet_statistics_keys = [
    "tweet_type",
    "view_count",
    "twitter_engagement_rate",
    "retweet_count",
    "reply_count",
    "reach",
    "quote_count",
    "influence",
    "impression_rate",
    "impression",
    "favorited",
    "favorite_count",
    "engagement_rate",
    "engagement",
]

def show_tweets(tweets, tweets_type, uid=None):
    tweets = [t for t in tweets if t['user']['id'] == uid]
    if not tweets:
        return
    print('-'*100)
    print(f'Number of {tweets_type}:', len(tweets))
    ts = [t['text'].replace('\u200c', '').split('\n') for t in tweets]
    
    user_statistics = dict(filter(
        lambda y: y[0] in user_statistics_keys,
        tweets[0]['user'].items()
        )
    )
    print(user_statistics)
    print('-'*100)

    for i, t in enumerate(ts):
        print('_'*100)
        for line in t:
            print(line)
    print('\n'*2)


In [ ]:
for (valid_uid, score) in sorted(scores, key=lambda x: -x[1]):
    print('','#'*100,'\n', f'User ID: {valid_uid}, scores: {score}\n', '#'*100,'\n')
    show_tweets(top_tweets, 'valid_tweets', valid_uid)



 #################################################################################################### 
 User ID: 1591547142136627202, scores: 837.4029193675249
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of valid_tweets: 19
{'name': 'علی اکبر رائفی پور', 'verified': True, 'followers_count': 61940, 'friends_count': 324, 'listed_count': 136, 'favourites_count': 1701, 'statuses_count': 2227}
----------------------------------------------------------------------------------------------------
____________________________________________________________________________________________________
حالا که قرار است با نفت تنهاتری (که البته بسیار کار پسندیده ای است ) زیرساخت ها را توسعه دهیم به نظرشما بین توسعه شهر فرودگاهی امام خمینی و یا توسعه ظرفیت های باری - مسافری ریلی کشور کدامیک اولویت دارد ؟
@mehrdadbazrpash
_________________

In [ ]:
users = []
for (valid_uid, score) in sorted(scores, key=lambda x: -x[1]):
    for t in top_tweets:
        tweets = None
        if t['user']['id'] == valid_uid:
            users.append(t['user'])
            break

pd.set_option('display.max_rows', None)
df = pd.DataFrame(users)
df.filter(user_statistics_keys)

,name,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count
0,علی اکبر رائفی پور,True,61940,324,136,1701,2227
1,سینامیم,False,69553,61782,17,196811,175113
2,یاسی,False,33813,395,13,79904,8854
3,Reza Pahlavi,True,1364301,72,1971,207,3545
4,ايران اينترنشنال,True,1697520,74,1900,42,222473
5,تک مصرع,False,34068,0,22,23389,2234
6,حنانیا نفتالی,True,31057,190,88,2380,5356
7,اسرائیل به فارسی,True,497687,361,932,23145,16248
8,سِباسْتین,False,76822,800,37,33970,33495
9,مجید نوری🇮🇷,False,4993,306,14,5921,2797
